In [52]:
from omdb import OMDBClient
import pandas as pd
import requests
import math, yaml
import numpy as np

In [53]:
df = pd.read_csv("needed_info.csv")

In [54]:
df.shape

(22588, 11)

## Year

In [56]:
#turning the year column into two seperate columns
df[['startYear', 'endYear']] = df.year.str.split("–", expand=True)
df.drop("year", inplace=True, axis=1)

In [57]:
df.head()

,id,title,total_seasons,imdb_votes,runtime,genre,country,writer,startYear,endYear
0,tt0039123,Kraft Theatre,11.0,98,60 min,Drama,USA,NaN,1947,1958
1,tt0040021,Actor's Studio,2.0,48,30 min,Drama,USA,NaN,1948,1950
2,tt0040036,The Chevrolet Tele-Theatre,2.0,24,30 min,Drama,USA,NaN,1948,1950
3,tt0040041,Texaco Star Theatre,8.0,70,60 min,"Comedy, Family",USA,NaN,1948,1956
4,tt0040042,The Morey Amsterdam Show,3.0,24,30 min,"Comedy, Family, Music",USA,NaN,1948,1950


## Collecting season ratings

In [59]:
def getSeasonAvg(imdb_id,numSeasons):
    
   
    #"e0a9acf2"
    id = "i=" + imdb_id
    url = "http://www.omdbapi.com/?"
    api_key="&apikey=e0a9acf2"
    season="&Season="
    
    #data = getNumSeasons(id)
    if(numSeasons != 'N/A'):
        numSeasons = int(numSeasons)
    
        #ratingsum=0
        seasonsRatings = []
        for i in range(numSeasons):
            seasoncurr = season + str(i+1)
            link2=url+id+seasoncurr+api_key
            resp=requests.get(link2)
            jsondata=resp.json()
        
            if(len(jsondata) > 2):
                episodes = jsondata['Episodes']
                num = len(episodes)
                
       
                ratingsum=0
                for i in range(len(episodes)):
                    if(episodes[i]['imdbRating'] == 'N/A'):
                        num = num-1
                    else:
                        ratingsum = ratingsum+float(episodes[i]['imdbRating'])
                
                if(num > 0):
                    ratingsum = round(ratingsum/num,1)
                else:
                    ratingsum=float("NaN")
            else:
                ratingsum=float("NaN")
                
            seasonsRatings.append(ratingsum)
        #print(imbd_id)
        return seasonsRatings

In [60]:
df['seasonRating']=" "

In [70]:
df['seasonRating'][:10000]=list(map(getSeasonAvg, df["id"][:10000], df["total_seasons"][:10000]))

C:\Users\oyino\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [72]:
df['seasonRating'][10000:15000]=list(map(getSeasonAvg, df["id"][10000:15000], df["total_seasons"][10000:15000]))

C:\Users\oyino\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [96]:
df['seasonRating'][15000:]=list(map(getSeasonAvg, df["id"][15000:], df["total_seasons"][15000:]))

C:\Users\oyino\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#change the ratings column from a list to a dictionary
def ratingsTodict(season):
    thisdict = {}
    if season is not None:
        for i in range(len(season)):
            seasons = "Season " + str(i+1) 
            thisdict[seasons] = season[i] 
    return thisdict

In [ ]:
df['seasonRating']=tuple(ratingsTodict(i) for i in df['seasonRating'])

In [ ]:
#change all string nan values to np.nan
def change_nan(thisdict):
    newdict=thisdict.copy()
    for x, y in newdict.items():
        if(y=="nan"):
            newdict[x]=np.nan
            
    return newdict

In [ ]:
df['seasonRating']=tuple(change_nan(i) for i in df['seasonRating'])

In [ ]:
df.head()

## Season Ratings

In [105]:
ratings=df["seasonRating"]
ratings = ratings.apply(pd.Series)
ratings.head()

,0,1,2,3,4,5,6,7,8,9,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.8,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,7.2,NaN,6.6,6.2,NaN,7.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
ratings[['id', 'title']]=df[['id', 'title']].copy()

In [115]:
#specifying order of the columns
cols = ['id'] + ['title'] + [col for col in ratings if col != 'id' and col!='title']
ratings = ratings[cols]

In [116]:
ratings.head()

,id,title,0,1,2,3,4,5,6,7,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,tt0039123,Kraft Theatre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0040021,Actor's Studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0040036,The Chevrolet Tele-Theatre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0040041,Texaco Star Theatre,NaN,NaN,7.2,NaN,6.6,6.2,NaN,7.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tt0040042,The Morey Amsterdam Show,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
#ratings.to_csv('raw_ratings.csv', index=False)

In [112]:
ratings.columns

Index([   'id', 'title',       0,       1,       2,       3,       4,       5,
             6,       7,
       ...
          2009,    2010,    2011,    2012,    2013,    2014,    2015,    2016,
          2017,    2018],
      dtype='object', length=2021)

In [111]:
#we will only be analysing up to 10 seasons
ratingsdf = ratings.drop(ratings.loc[:, 'Season 11': ].columns, axis = 1)

KeyError: '11'

In [78]:
ratingsdf.head()

,id,title,Season 1,Season 2,Season 3,Season 4,Season 5,Season 6,Season 7,Season 8,Season 9,Season 10
0,tt0039123,Kraft Theatre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.8,8.4,NaN
1,tt0040021,Actor's Studio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0040036,The Chevrolet Tele-Theatre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0040041,Texaco Star Theatre,NaN,NaN,7.2,NaN,6.6,6.2,NaN,7.6,NaN,NaN
4,tt0040042,The Morey Amsterdam Show,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
ratingsdf.shape

(22588, 11)

In [45]:
ratingsdf.head()

,id,Season 1,Season 2,Season 3,Season 4,Season 5,Season 6,Season 7,Season 8,Season 9,Season 10
0,tt0039123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.8,8.4,NaN
1,tt0040021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0040036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0040041,NaN,NaN,7.2,NaN,6.6,6.2,NaN,7.6,NaN,NaN
4,tt0040042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
##removing rows that have np.nan in every columns
removed=[]
#for each row in the dataframe
for i, val in ratingsdf.iterrows():
    valid = 0
    for col in ratingsdf:
        if( col != 'id' and col!='title'):
            if (ratingsdf[col][i] ==0.0 or math.isnan(ratingsdf[col][i])):
                valid =valid+1 # add one for each column with a nan value
    
    if(valid ==10):
        #store the index of rows to be removed
        removed.append(i)

In [47]:
len(removed)

1870

In [ ]:
ratingsdf=ratingsdf.drop(removed)

In [ ]:
ratingsdf.reset_index(inplace=True, drop=True)

In [ ]:
ratingsdf.to_csv("ratings.csv", index=False)

In [127]:
df=df.drop(removed)

In [128]:
df.reset_index(inplace=True, drop=True)

## Runtime

In [ ]:
df.shape

In [132]:
df['runtime']=[(df['runtime'].loc[i].split(" ")[0])  if isinstance(df['runtime'].loc[i], str) else df['runtime'][i] for i in df.index]

In [133]:
#one row has a comma which stops us from turning the string to an int
a=df['runtime'].loc[730]
print(a)

In [135]:
#remove this comma
my_float = float(a.replace(',', ''))

In [136]:
df['runtime'].loc[730]=my_float

In [137]:
#strung to numeric
df['runtime']=pd.to_numeric(df['runtime'])

## Country

In [145]:
countrydf=df[['id', 'title']].copy()

In [146]:
countrydf['Country']=df['country'].copy()

In [147]:
#turn string to a list
countrydf['Country']=[str(value).split(",") for value in countrydf['Country']]

In [148]:
coun=[]

In [149]:
for i, value in countrydf['Country'].items():
    for i in value:
        if (i not in coun):
            coun.append(i)

In [150]:
coun = [x.strip(' ') for x in coun]

In [151]:
print(coun)

["['USA']", "['Canada'", "' USA']", "['USA'", "' UK']", "['UK']", "['West Germany']", "' Australia'", "' UK'", "' Canada']", "['Australia']", "' Mexico']", "['Yugoslavia']", "['Canada']", "['Denmark']", "['Netherlands']", "['UK'", "' Italy']", "['Sweden']", "['France'", "['West Germany'", "' Germany']", "' Australia']", "' Belgium']", "['Australia'", "['Japan']", "['Germany'", "' Austria'", "' Switzerland']", "' Hungary'", "' Austria']", "' Norway'", "' Denmark'", "' France']", "' France'", "' Canada'", "' Japan'", "' Taiwan']", "' Japan']", "' Belgium'", "' Philippines'", "' South Korea']", "' USA'", "' New Zealand']", "['Spain'", "' Italy'", "' Germany'", "' Taiwan'", "' South Korea'", "' China'", "' Cuba'", "' Switzerland'", "' West Germany'", "' Poland'", "' Bulgaria'", "' Portugal'", "' Yugoslavia']", "['Netherlands'", "' West Germany']", "['Spain']", "['Switzerland'", "['Germany']", "['Sweden'", "' Finland']", "['France']", "['Norway']", "' Spain']", "' Hungary']", "['Austria'", 

In [152]:
#remove duplicate country values
coun = list(dict.fromkeys(coun))

In [ ]:
print(coun)

In [ ]:
for i in coun:
    countrydf[i] = " "

In [ ]:
for i, value in countrydf["Country"].items():
    valid = False;
    for k in value:
        k=k.strip()
        for j in range(len(coun)):
            if(k == coun[j]):
                countrydf[coun[j]][i] = "true"
            else:
                " "

In [ ]:
#Turning three columns West Germany, East Germany and Germany into 1
countrydf['Germany'] = ["true" if i=="true" or j=="true" or k=="true" else " " for i,j,k in zip(countrydf['West Germany'], countrydf['Germany'], countrydf['East Germany']) ]

In [ ]:
countrydf.drop(["West Germany", "East Germany"], axis=1, inplace=True)

In [ ]:
countrydf.to_csv("country.csv", index=False)

In [ ]:
df['country']=genredf['Country'].copy()

## Genre

In [94]:
genredf=df[['id', 'title']].copy()

In [95]:
genredf['Genre']=df['genre'].copy()

,id,title
0,tt0039123,Kraft Theatre
1,tt0040021,Actor's Studio
2,tt0040036,The Chevrolet Tele-Theatre
3,tt0040041,Texaco Star Theatre
4,tt0040042,The Morey Amsterdam Show
5,tt0040048,Perry Como's Kraft Music Hall
6,tt0040049,The Philco-Goodyear Television Playhouse
7,tt0040051,Studio One in Hollywood
8,tt0040053,The Ed Sullivan Show
9,tt0041005,Believe It or Not


In [ ]:
#turn string to a list
genredf['Genre']=[str(value).split(",") for value in genredf['Genre']]

In [ ]:
genredf.head()

In [ ]:
#get all values in a value
genlist = []
for i, value in genredf['Genre'].items():
    for i in value:
        if (i not in genlist):
            genlist.append(i)

In [ ]:
genlist

There are duplicate country's because of leading & trailing space

In [ ]:
#strip all values so duplicate country's can be identified
genlist = [x.strip(' ') for x in genlist]

In [ ]:
#remove duplicate country names
genlist = list(dict.fromkeys(genlist))

In [ ]:
for i in genlist:
    genredf[i] = " "

In [ ]:
for i, value in genredf["Genre"].items():
    valid = False;
    for k in value:
        k=k.strip()
        for j in range(len(genlist)):
            if(k ==genlist[j]):
                genredf[genlist[j]][i] = "true"
            else:
                " "

In [ ]:
df['genre']=genredf['Genre'].copy()

In [ ]:
genredf.to_csv("genre.csv", index=False)

In [ ]:
df.to_csv("all_data.csv", index=False)